In [24]:
print("******INSTALLING ESSENTIAL PACKAGES*****")
import sys
!{sys.executable} -m pip install firebase
!{sys.executable} -m pip install python_jwt
!{sys.executable} -m pip install gcloud
!{sys.executable} -m pip install pycryptodome
!{sys.executable} -m pip install requests-toolbelt
!{sys.executable} -m pip install sseclient
print("\n\n\n\n\n*****PACKAGES SUCCESSFULLY INSTALLED*****")
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from firebase import firebase

error_type="null"
strf="Empty"
firebase=firebase.FirebaseApplication("https://first-firebase-app-f7863.firebaseio.com/",None)
#This function returns movie title coreesponding to its index value
def get_title_from_index(index):
	return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
	return df[df.title == title]["index"].values[0]

#Read CSV File
df = pd.read_csv("C:\\Users\\Satish Pratap Singh\\Documents\\movie_dataset.csv")
#print df.columns
#Procedure to Select Features

features = ['keywords','cast','genres','director']
#Create a column in DF which combines all selected features
for feature in features:
	df[feature] = df[feature].fillna('')

def combine_features(row):
	try:
		return row['keywords'] +" "+row['cast']+" "+row["genres"]+" "+row["director"]
	except:
		print("Error:",row)	

df["combined_features"] = df.apply(combine_features,axis=1)

#print "Combined Features:", df["combined_features"].head()

#Create count matrix from this new combined column
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])

#Compute the Cosine Similarity based on the count_matrix
cosine_sim = cosine_similarity(count_matrix)
running_user=firebase.get('/',"running")
movie_user_likes =firebase.get("/"+running_user,"interested_movie")
bound_value=firebase.get("/"+running_user,"bound_value")

#Get index of this movie from its title
try:
    movie_index = get_index_from_title(movie_user_likes)
    similar_movies =  list(enumerate(cosine_sim[movie_index]))
    #Get a list of similar movies in descending order of similarity score
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

    #Print titles of first atmost 50 movies
    i=0
    strf=""
    for element in sorted_similar_movies:
        if i==0:
            i=1
            get_title_from_index(element[0])
            continue
        if i==int(bound_value):
            strf=strf+str(i)+"==>"+str(get_title_from_index(element[0]))
        else:    
            strf=strf+str(i)+"==>"+str(get_title_from_index(element[0]))+" \n"
        if i>=int(bound_value):
            break
        i=i+1  
except IndexError:
    error_type="Movie Name is not in the Database"
firebase.put("9711366786/","likes",strf)
firebase.put("/","mode",1)
firebase.put("/","error",error_type)
if error_type!="null":
    print("|n\n*****SYSTEM CAN NOT RECOMMEND BECAUSE OF:",error_type+" ******")
else:
    print("\n\n*****MOVIE LIST UPDATED SUCCESSFULLY*****")

******INSTALLING ESSENTIAL PACKAGES*****





*****PACKAGES SUCCESSFULLY INSTALLED*****


*****MOVIE LIST UPDATED SUCCESSFULLY*****
